In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import MSELoss
from torchvision import transforms
from torch.utils.data import DataLoader

from custom_dataset import CustomDataset

In [2]:
# Prepare paths
data_path = '../data/'
processed_data_path = os.path.join(data_path, 'processed')

# Read personal data from excel
personal_data = pd.read_excel(os.path.join(data_path, 'PersonalData.xlsx'))

# Read the data that is the result of the converted videos
data = pd.read_csv(os.path.join(processed_data_path, 'AllSquats.csv'))

# Merge personal and video data
data = pd.merge(data, personal_data, on='Id')

In [3]:
# Calculate the maximum load that was passed
max_load = data.loc[data['Lifted'] == 1, ['Id', 'Load']].groupby(by='Id', as_index=False).max()
max_load = max_load.rename(columns={'Load': 'MaxLoad'})
data = pd.merge(data, max_load, on='Id')

# Calculate what percentage of the maximum load is the current load
data['PercentageMaxLoad'] = data['Load'] / data['MaxLoad']

del data['MaxLoad']


# Get only lifted approaches
data = data.loc[data['Lifted'] == 1]

# Variables that aren't needed in the first run
to_drop = [
    'Id', 'Age', 'Height', 'Weight', 'PastInjuries', 'LastInjury', 'PainDuringTraining', 'SquatRecord',
    'BenchPressRecord', 'DeadliftRecord', 'PhysicalActivities', 'SetNumber', 'Load', 'Lifted', 'Timestamp']

data = data.drop(columns=to_drop)

# Categorical variables that need to be one hot encoded
to_one_hot = [
    'ProficiencyLevel', 'EquipmentAvailability', 'TrainingProgram', 'TrainingFrequency', 'CameraPosition']

dataframe = pd.get_dummies(data, columns=to_one_hot, dtype=int)

# Move the PercentageMaxLoad column to the end of the dataframe
percentage = dataframe.pop('PercentageMaxLoad')
dataframe['PercentageMaxLoad'] = percentage

In [4]:
# Get unique file IDs
file_ids = dataframe['FileId'].unique()

# Split the files into three lists in an 8:1:1 ratio
train_ids, ids_to_split = train_test_split(file_ids, test_size=0.2)

valid_ids, test_ids = train_test_split(ids_to_split, test_size=0.5)

In [5]:
class Norm(object):
    """
    
    """
    def __call__(self, tensor):
        #
        min_value = tensor.min()
        max_value = tensor.max()

        # Normalization procedure
        normalized = 2 * (tensor - min_value) / (max_value - min_value) - 1

        return normalized

In [6]:
batch_size = 16
num_workers = 1
pin_memory = False

In [7]:
train_dataset = CustomDataset(
    dataframe.loc[dataframe['FileId'].isin(train_ids)],
    transform=Norm()
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    # num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=True
)

valid_dataset = CustomDataset(
    dataframe.loc[dataframe['FileId'].isin(valid_ids)],
    transform=Norm()
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=False
)

test_dataset = CustomDataset(
    dataframe.loc[dataframe['FileId'].isin(test_ids)],
    transform=Norm()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=False
)

In [8]:
class CustomRNN(nn.Module):
    """

    """
    def __init__(self, input_size, hidden_size, device='cuda', dtype=torch.float64):
        super(CustomRNN, self).__init__()
        # Initialize 
        self.hidden_size = hidden_size
        self.device = device

        # Initialize input to hidden state layer
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size, dtype=dtype)
        # Initialize hidden state to output layer
        self.fc = nn.Linear(hidden_size, 1, dtype=dtype)


    def init_hidden_state(self):
        """
        
        """
        return torch.zeros(1, self.hidden_size, device=self.device)


    def forward(self, input_tensor, hidden_tensor):
        """
        
        """
        # Concatenate input and hidden tensors
        combined = torch.cat((input_tensor.unsqueeze(dim=0), hidden_tensor), 1)

        hidden = self.i2h(combined)
        output = self.fc(hidden)

        return hidden, output

In [9]:
sample = next(iter(train_loader))[0]
sample.shape

torch.Size([16, 238, 78])

In [10]:
input_size = 78
hidden_size = 128
output_size = 1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float64

learning_rate = 0.001

model = CustomRNN(input_size=78, hidden_size=128)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = MSELoss()

In [12]:
# Put model in train mode
model.train()

# 
accumulated_loss = 0.

for batch_idx, (data, targets) in enumerate(train_loader):
    # Send data to target device
    data, targets = data.to(device), targets.to(device)

    # Forward pass
    # Prepare predictions storage
    predictions = torch.tensor([], device=device)

    for sample in data:
        # Initialize hidden state tensor 
        hidden_tensor = model.init_hidden_state()

        for frame in sample:
            hidden_tensor, output = model(frame, hidden_tensor)
        
        # Concatenate all predictions per batch
        predictions = torch.cat((predictions, output), dim=0)

    # Calculate and accumulate loss
    loss = loss_fn(targets, predictions)
    accumulated_loss += loss.item()

    # Optimizer zero grad
    optimizer.zero_grad()

    # Loss backward
    loss.backward()

    # Optimizer step
    optimizer.step()

    print(f"batch: {batch_idx}, loss: {loss}")

# Get average loss per batch
accumulated_loss = accumulated_loss / len(train_loader)

batch: 0, loss: 0.13243898084199185
batch: 1, loss: 0.14506942547706375
batch: 2, loss: 0.09656047494274622
batch: 3, loss: 0.07118154356547206
batch: 4, loss: 0.05537154544152028
batch: 5, loss: 0.09846964176617123
batch: 6, loss: 0.09731892511120008
batch: 7, loss: 0.07409425919017369
batch: 8, loss: 0.09112772576331318
batch: 9, loss: 0.06318715488308507
batch: 10, loss: 0.05450714218489702
batch: 11, loss: 0.07325507025877741
batch: 12, loss: 0.12059554791427138
batch: 13, loss: 0.1981998602999847
batch: 14, loss: 0.11003481937002285
batch: 15, loss: 0.0599297699780742
batch: 16, loss: 0.07371528910697367
batch: 17, loss: 0.043689598251130994
batch: 18, loss: 0.10735060160091554
batch: 19, loss: 0.1328117789806159
batch: 20, loss: 0.06740839998347338
batch: 21, loss: 0.051677960514256625
batch: 22, loss: 0.05263675199993149
batch: 23, loss: 0.07053165445901295
batch: 24, loss: 0.07229346493866429
batch: 25, loss: 0.04623695908584583
batch: 26, loss: 0.07151154544336578
batch: 27, l

In [13]:
accumulated_loss

0.07107688258760203